In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import cross_val_score
import sklearn
from sklearn import metrics
import time
import pickle
from keras.models import load_model
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

c:\apps\anaconda2\envs\env2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Read data

In [8]:
names = ['Toxic', 'Severe', 'Obscene', 'Threat', 'Insult', 'Identity']

In [176]:
df = pd.read_csv('spell_checker32MMp1.csv')
df = pd.concat([df, pd.read_csv('spell_checker32MMp2.csv')])
df = pd.concat([df, pd.read_csv('spell_checker32MMp3.csv')])
df = df.iloc[:, 1:]
df = df.reset_index(drop=True)

ids = pd.read_csv('test.csv')
ids = ids['id']

In [11]:
xx = df.as_matrix()
del df
xx

array([[-0.05647484,  0.0943086 ,  0.18976713, ...,  0.00348375,
         0.00067335, -0.01066972],
       [ 0.16755086, -0.11362292, -0.20672155, ..., -0.06140746,
        -0.03262367, -0.00359173],
       [ 0.09340793, -0.09266319, -0.13996158, ..., -0.0206565 ,
         0.01268927, -0.00550548],
       ...,
       [-0.10872087,  0.00290806,  0.02554503, ..., -0.01376142,
        -0.00544102,  0.00915878],
       [-0.10625417,  0.0177638 , -0.11193075, ...,  0.01413548,
        -0.01402142,  0.00892577],
       [ 0.30270611,  0.17246708, -0.01458978, ..., -0.01239552,
         0.02174533,  0.01650812]])

### Load the LR classifiers

In [12]:
path = 'clfs/LR/'
LR = {}
for name in names:
    filename = "LR_{name}.sav".format(name=name)
    LR[name] = pickle.load(open(path + filename, 'rb'))


In [13]:
LR

{'Toxic': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=1, warm_start=False),
 'Severe': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=1, warm_start=False),
 'Obscene': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
           penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
           verbose=1, warm_start=False),
 'Threat': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=100, multi_class='ovr', n_jo

### Load the LDA classifiers

In [14]:
path = 'clfs/LDA/'
LDA = {}
for name in names:
    filename = "LDA_{name}.sav".format(name=name)
    LDA[name] = pickle.load(open(path + filename, 'rb'))


In [15]:
LDA

{'Toxic': LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
               solver='svd', store_covariance=False, tol=0.0001),
 'Severe': LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
               solver='svd', store_covariance=False, tol=0.0001),
 'Obscene': LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
               solver='svd', store_covariance=False, tol=0.0001),
 'Threat': LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
               solver='svd', store_covariance=False, tol=0.0001),
 'Insult': LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
               solver='svd', store_covariance=False, tol=0.0001),
 'Identity': LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
               solver='svd', store_covariance=False, tol=0.0001)}

### Load MLPs

In [16]:
# load model from file
path = 'clfs/MLP/'
MLP = {}
for name in names:
    filename = "MLP_{name}.h5".format(name=name)
    MLP[name] = load_model(path + filename)


In [17]:
MLP

{'Toxic': <keras.models.Sequential at 0x1e998a9fac8>,
 'Severe': <keras.models.Sequential at 0x1e9cd373710>,
 'Obscene': <keras.models.Sequential at 0x1e998c8eb38>,
 'Threat': <keras.models.Sequential at 0x1e998eee7f0>,
 'Insult': <keras.models.Sequential at 0x1e9cea13978>,
 'Identity': <keras.models.Sequential at 0x1e999650320>}

### Predict probabilities

In [177]:
dp = pd.DataFrame(columns=['id'])
dp['id'] = ids

In [183]:
dm = pd.DataFrame(columns=['id', 'Toxic', 'Severe', 'Obscene', 'Threat', 'Insult', 'Identity'])
dm['id'] = ids

def classify_comment(matrix):
    global dm
    for i, row in enumerate(matrix):
        row = np.atleast_2d(row)
        for name in names:
            dm[name].iloc[i] = ((MLP[name].predict(row) + LR[name].predict_proba(row)[0, 1] + LDA[name].predict_proba(row)[0, 1]) / 3.0)[0][0]
#             print('MLP output', MLP[name].predict(row))
#             print('LR output', LR[name].predict_proba(row))
#             print('LDA output', LDA[name].predict_proba(row))

def classify(vector):
    print(vector)
    
classify_comment(xx[:100, :])
dm.head(100)

,id,Toxic,Severe,Obscene,Threat,Insult,Identity
0,00001cee341fdb12,0.903428,0.792703,0.99656,0.982161,0.969188,0.712949
1,0000247867823ef7,0.0359404,0.13601,0.53579,0.0432466,0.0642054,0.437309
2,00013b17ad220c46,0.0136301,0.999234,0.49967,0.161457,0.055157,0.202645
3,00017563c3f7919a,0.00918823,0.9829,0.111489,0.131479,0.00966507,0.970891
4,00017695ad8997eb,0.86128,0.830417,0.779917,0.962902,0.921139,0.984436
5,0001ea8717f6de06,0.68906,0.569941,0.999969,0.999896,0.984372,0.944233
6,00024115d4cbde0f,0.344618,0.16343,0.8963,0.926384,0.762595,0.686477
7,000247e83dcc1211,0.889868,0.657191,0.575847,0.255776,0.620129,0.612536
8,00025358d4737918,0.421598,0.0936823,0.451597,0.0920589,0.388948,0.51862
9,00026d1092fe71cc,0.00631172,0.749616,0.818244,0.0652476,0.175904,0.33605


In [165]:
xx.shape

(153164, 1000)

In [191]:
# toxic
tmp_labels = ['Toxic', 'Severe', 'Obscene', 'Threat', 'Insult', 'Identity']
lables = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
classifiers = {lables[idx]:[MLP[labe], LDA[labe], LR[labe]] for idx, labe in enumerate(tmp_labels)}
ignored_models = [0, 2]

final_table = dp
for i, label in enumerate(lables):
    dp1 = dp[:100]
    label_classifiers = classifiers[label]
    c = 0
    for idx, classifer in enumerate(label_classifiers):
        if idx in ignored_models:
            continue
        c += 1
        if idx == 0:
            dp1 = pd.concat([dp1, pd.DataFrame(classifer.predict(xx))], axis=1)
        else:
            dp1 = pd.concat([dp1, pd.DataFrame(classifer.predict_proba(xx)[:, 1])], axis=1)
    dp1[label] = dp1.iloc[:, 1:].sum(axis=1) / c
    final_table = pd.concat([final_table, dp1[label]], axis=1)
final_table.to_csv("final_fisher_lda.csv", index= False)

In [189]:
final_table.shape

(153164, 7)

In [120]:
print(xx[:10, :].shape)
classify_comment(xx[:, :])

(10, 1000)

In [ ]:
dp.to_csv('solution.csv')